Given the signature of a path, we extend its signature with new incoming data with Chen's relation and shorten the signature (i.e. calculate the signature of the underlying path without the first data point) again with Chen's relation via reversing the initial piece of the path which should be removed from its signature.

In [1]:
import torch
from signatory import signature, signature_combine
import numpy

channels, sig_depth = 2, 4
path_full = torch.rand(5, channels)
path_given = path_full[:-1,:]
path_final = path_full[1:,:]
path_new = path_full[-1:,:]
path_old = path_full[:2,:]
path_old_reversed = torch.flipud(path_old)

print_paths = False
if print_paths:
    print(path_full)
    print(path_given)
    print(path_final)
    print(path_new)
    print(path_old)


sig_path_given = signature(path_given.unsqueeze(0), sig_depth)
sig_path_full_updated = signature(path_new.unsqueeze(0), sig_depth, 
                                  basepoint=path_given[-1,:].unsqueeze(0), 
                                  initial=sig_path_given)
print('Check:', numpy.isclose(sig_path_full_updated,
                             signature(path_full.unsqueeze(0), sig_depth)))
sig_path_old_reversed = signature(path_old_reversed.unsqueeze(0), sig_depth)
sig_path_final_combined = signature_combine(sig_path_old_reversed, sig_path_full_updated,
                                           channels, sig_depth)

print('Check:', numpy.isclose(sig_path_final_combined, 
                              signature(path_final.unsqueeze(0), sig_depth)))






Check: [[ True  True  True  True  True  True  True  True  True  True  True  True
   True  True  True  True  True  True  True  True  True  True  True  True
   True  True  True  True  True  True]]
Check: [[ True  True  True  True  True  True  True  True  True  True  True  True
   True  True  True  True  True  True  True  True  True  True  True  True
   True  True  True  True  True  True]]


In [ ]:

path = torch.rand(10, channels)
print(path)
path_remaining = path[5:,:]
print(path_remaining)
path_short = path[0:6,:]
print(path_short)
path_short_reversed = torch.flipud(path_short)
print(path_short_reversed)




## Version 1 with concatenated path
concatenated_path = torch.cat([path_short_reversed, path],dim=0)
print(concatenated_path)
sig_conc_path = signatory.signature(concatenated_path.unsqueeze(0), sig_depth)
print(sig_conc_path)

## Version 2 with singature combine
sig_path = signatory.signature(path.unsqueeze(0), sig_depth)
sig_short_reversed = signatory.signature(path_short_reversed.unsqueeze(0), sig_depth)
sig_combined_path = signatory.signature_combine(sig_short_reversed, sig_path, channels, sig_depth)
print(sig_combined_path)

## Control: Signature of remaining path only
sig_path_remaining = signatory.signature(path_remaining.unsqueeze(0), sig_depth)
print(sig_path_remaining-sig_conc_path)
print(numpy.isclose(sig_path_remaining, sig_combined_path))
print(numpy.isclose(sig_path_remaining, sig_conc_path))


Test with SigQfunction methods


In [15]:
%run base.ipynb
%load_ext autoreload
%autoreload 2
from qlearning_policies import SigQFunction
from signatory import signature
import torch
import numpy

channels, sig_depth = 2, 4

path_full = torch.rand(5, channels)
path_given = path_full[:-1,:]
path_final = path_full[1:,:]
path_new = path_full[-1:,:]
path_old = path_full[:2,:]
path_old_reversed = torch.flipud(path_old)

                                      
print('Check initial signature computation:')
print('without basepoint:')
sigQfunction = SigQFunction(in_channels=channels, out_dimension=3, 
                            sig_depth=sig_depth, initial_basepoint=None)
print(numpy.isclose(signature(path_full.unsqueeze(0), depth=sig_depth),
                    sigQfunction.calculate_path_signature(path_full.unsqueeze(0))))

print('with basepoint:')
basepoint=[-0.65, 1.]
sigQfunction = SigQFunction(in_channels=channels, out_dimension=3, 
                            sig_depth=sig_depth, initial_basepoint=basepoint)
print(numpy.isclose(
    signature(
        path_full.unsqueeze(0), 
        depth=sig_depth, 
        basepoint=torch.tensor(basepoint, requires_grad=False).unsqueeze(0)
    ),
    sigQfunction.calculate_path_signature(path_full.unsqueeze(0))
))


print('Check signature update:')
print('without basepoint:')
sigQfunction = SigQFunction(in_channels=channels, out_dimension=3, 
                            sig_depth=sig_depth, initial_basepoint=None)

sig_path_given = sigQfunction.calculate_path_signature(path_given.unsqueeze(0))
sig_path_updated = sigQfunction.update_path_signature(path_new.unsqueeze(0), 
                                                      basepoint=path_given.unsqueeze(0)[:,-1,:], 
                                                      signature=sig_path_given)
print(numpy.isclose(sig_path_updated, signature(path_full.unsqueeze(0), depth=sig_depth)))

print('with basepoint:')
basepoint=[-0.65, 1.]
sigQfunction = SigQFunction(in_channels=channels, out_dimension=3, 
                            sig_depth=sig_depth, initial_basepoint=basepoint)
sig_path_given = sigQfunction.calculate_path_signature(path_given.unsqueeze(0))
sig_path_updated = sigQfunction.update_path_signature(path_new.unsqueeze(0), 
                                                      basepoint=path_given.unsqueeze(0)[:,-1,:], 
                                                      signature=sig_path_given)
print(numpy.isclose(sig_path_updated, 
                    signature(path_full.unsqueeze(0), 
                              depth=sig_depth,
                              basepoint=torch.tensor(basepoint, requires_grad=False).unsqueeze(0)
)))

print('Check signature shorting:')
print('without basepoint:')
sigQfunction = SigQFunction(in_channels=channels, out_dimension=3, 
                            sig_depth=sig_depth, initial_basepoint=None)



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Check initial signature computation:
without basepoint:
[[ True  True  True  True  True  True  True  True  True  True  True  True
   True  True  True  True  True  True  True  True  True  True  True  True
   True  True  True  True  True  True]]
with basepoint:
[[ True  True  True  True  True  True  True  True  True  True  True  True
   True  True  True  True  True  True  True  True  True  True  True  True
   True  True  True  True  True  True]]
Check signature update:
without basepoint:
[[ True  True  True  True  True  True  True  True  True  True  True  True
   True  True  True  True  True  True  True  True  True  True  True  True
   True  True  True  True  True  True]]
with basepoint:
[[ True  True  True  True  True  True  True  True  True  True  True  True
   True  True  True  True  True  True  True  True  True  True  True  True
   True  True  True  True  True  True]]
Check signature shorting:
